# Questionnaire to Analysis Correlation Experiment

This notebook demonstrates how to correlate **Homeowner Interview Answers** with **Architecture Analysis** to generate personalized, high-value suggestions.

## Goal
Prove that explicitly instructing the LLM to cross-reference interview answers results in better, context-aware suggestions compared to a baseline analysis.

In [ ]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# 1. Setup & Auth
# Load .env from backend root
env_path = Path("../.env")
if not env_path.exists():
    env_path = Path("../../.env")
load_dotenv(dotenv_path=env_path)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    print("WARNING: OPENAI_API_KEY not found. Please ensure .env is set.")
else:
    print("Environment loaded. Ready to run.")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

## 2. Mock Data Setup

In [ ]:
# A. Mock Architecture Context (Simulating what we extracted from a plan)
mock_plan_context = """
Project Address: 123 Maple Dr, Austin, TX
Total SqFt: 2800
House Type: Single Family
Exterior: Stucco with large glass windows on West facing facade.
Roof: Flat roof, TPO membrane.
Rooms:
- Master Bed: 18x16, large windows facing street.
- Home Office: 10x12, adjacent to Kitchen, sliding barn door.
- Kitchen: Open concept, large island.
"""

# B. Mock Interview Answers (The user's specific constraints)
mock_answers = {
    "privacy_concern": "High. We value privacy significantly and do not want street visibility.",
    "noise_sensitivity": "Very Sensitive. Use the home office for recording, need silence.",
    "budget_tier": "Economy. We want to save money where possible.",
    "natural_light": "Love natural light, but hate glare."
}

print("Mock Data Ready.")

## 3. Baseline Analysis (Current State)
Analyzes the plan *without* explicit reference to the interview answers.

In [ ]:
baseline_system_prompt = """
You are Buildwise AI. Analyze the provided architectural context and return a JSON object with a list of 'suggestions' for improvements.
Focus on standard architectural best practices.
Output JSON format: { "suggestions": ["string"] }
"""

baseline_messages = [
    SystemMessage(content=baseline_system_prompt),
    HumanMessage(content=f"Architecture Context:\n{mock_plan_context}")
]

print("Running Baseline Analysis...")
baseline_res = llm.invoke(baseline_messages)
try:
    baseline_data = json.loads(baseline_res.content)
    print("\n--- Baseline Suggestions ---")
    for s in baseline_data.get("suggestions", []):
        print(f"- {s}")
except Exception as e:
    print("Error parsing baseline:", e)
    print(baseline_res.content)

## 4. Correlated Analysis (Proposed State)
Analyzes the plan *with* the interview answers and an updated prompt instruction.

In [ ]:
correlated_system_prompt = """
You are Buildwise AI. Analyze the architectural context to provide personalized layout and material suggestions.

CRITICAL INSTRUCTION:
You have access to 'Homeowner Interview Answers'. You MUST cross-reference these answers with the floor plan features.
For every suggestion, if it relates to a user answer, explicitly mention *why* based on their answer.

Output JSON format:
{
  "suggestions": ["string"],
  "correlated_suggestions": [
    { "suggestion": "string", "reasoning": "string referencing specific user answer" }
  ]
}
"""

user_input = f"""
Architecture Context:
{mock_plan_context}

Homeowner Interview Answers:
{json.dumps(mock_answers, indent=2)}
"""

correlated_messages = [
    SystemMessage(content=correlated_system_prompt),
    HumanMessage(content=user_input)
]

print("Running Correlated Analysis...")
correlated_res = llm.invoke(correlated_messages)
try:
    correlated_data = json.loads(correlated_res.content)
    
    print("\n--- Correlated Suggestions (Personalized) ---")
    for item in correlated_data.get("correlated_suggestions", []):
        print(f"Suggestion: {item['suggestion']}")
        print(f"  -> Why: {item['reasoning']}")
        print("-")
        
    print("\n--- General Suggestions ---")
    for s in correlated_data.get("suggestions", []):
        print(f"- {s}")

except Exception as e:
    print("Error parsing correlated:", e)
    print(correlated_res.content)

## 5. Conclusion
Compare the "Baseline" vs "Correlated" output above.

**Expected Outcome:**
- **Baseline**: Might suggest generic things like "Consider energy efficient windows".
- **Correlated**: Should specifically say "Install clerestory windows or privacy film in Master Bed because you mentioned high privacy concern" or "Add solid core door and sound insulation to Home Office due to noise sensitivity for recording".